In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from functions import VM
import os
import numpy as np

# Take the sign names in our collections data

In [3]:
signs = os.listdir("Model_data") # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

# Create the our x and y data


In [16]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y train
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [13]:
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))